In [103]:
import math
from enum import Enum
import logging

In [130]:
SRC = 0 # 0:DRAM, 1:TCM
ACLK = 480
LATENCY = 125 # DDR4-2666:125
BUS_WIDTH = 32 # byte unit
OSTD = 4
BURST = 64 * 4 # byte unit

In [105]:
NCLK = 600
THROUGHPUT = 6 # byte unit

In [106]:
class DMAStatus(Enum):
    IDLE = 0
    REQ = 1
    GRANT = 2
    FULL = 3
    CONSUME = 4

In [107]:
throughput_a = THROUGHPUT * NCLK / ACLK

In [128]:
class DMA_FIFO:
    """DMA FIFO"""
    def __init__(self):
        self.status = DMAStatus.IDLE
        self.r_len = 0
        self.r_cnt = 0
        self.g_len = 0
        self.g_cnt = 0
        self.c_len = 0
        self.c_cnt = 0

    def set_req(self, length):
        self.r_len = LATENCY
        self.r_cnt = 0
        self.g_len = math.ceil(length / BUS_WIDTH) if SRC==1 else math.ceil(length / BUS_WIDTH) * 2 # somehow DRAM only got 1 cycle data every 2 cycle
        self.c_len = length
        self.status = DMAStatus.REQ

    def req_1t(self):
        logging.debug(str(self.r_cnt))
        self.r_cnt = self.r_cnt + 1
        if self.r_cnt >= self.r_len:
            self.set_grant()

    def set_grant(self):
        self.status = DMAStatus.GRANT
        self.g_cnt = 0

    def grant_1t(self):
        logging.debug(str(self.g_cnt))
        self.g_cnt = self.g_cnt + 1
        if self.g_cnt >= self.g_len:
            self.status = DMAStatus.FULL

    def set_consume(self):
        self.status = DMAStatus.CONSUME
        self.c_cnt = 0

    def consume_1t(self):
        logging.debug(str(self.c_cnt))
        self.c_cnt = self.c_cnt + throughput_a
        if self.c_cnt >= self.c_len:
            self.status = DMAStatus.IDLE

    def update(self, grant_ok):
        if self.status is DMAStatus.REQ:
            self.req_1t()
        elif self.status is DMAStatus.GRANT and grant_ok:
            self.grant_1t()
        elif self.status is DMAStatus.CONSUME:
            self.consume_1t()

In [117]:
class CMDGen:
    def __init__(self, burst, width, lines):
        self.burst = burst
        self.width = width
        self.lines = lines
        self.w_cntd = width
        self.l_cnt = 0
    def get_cmd(self):
        if self.l_cnt == self.lines:
            return 0
        if self.w_cntd <= self.burst:
            cmd = self.w_cntd
            self.w_cntd = self.width
            self.l_cnt = self.l_cnt + 1
            return cmd
        else:
            self.w_cntd = self.w_cntd - self.burst
            return self.burst
        

In [151]:
width = 96 * 4 # byte unit 
lines = 64

In [152]:
logging.basicConfig(filename='dma.log', filemode='w', level=logging.DEBUG)

DMAs = []
for i in range(OSTD):
    DMAs.append(DMA_FIFO())

cmdgen = CMDGen(burst=BURST, width=width, lines=lines)

req_idx = 0
grant_idx = 0
consume_idx = 0
cycle = 0
rdy_cycle = 0

cmd = cmdgen.get_cmd()
while True:
    logging.debug('cycle: ' + str(cycle))
    cycle = cycle + 1

    logging.debug('req_idx: ' + str(req_idx))
    logging.debug('grant_idx: ' + str(grant_idx))
    logging.debug('consume_idx: ' + str(consume_idx))

    if cmd != 0 and DMAs[req_idx].status is DMAStatus.IDLE:
        DMAs[req_idx].set_req(cmd)
        cmd = cmdgen.get_cmd()
        # print(cmd)
        if req_idx == OSTD - 1:
            req_idx = 0
        else:
            req_idx = req_idx + 1
            
    if DMAs[consume_idx].status is DMAStatus.FULL:
        DMAs[consume_idx].set_consume()

    consume_nxt = False
    all_idle = True
    dma_rdy = False
    for i, dma in enumerate(DMAs):    
        logging.debug(str(i) + '---' + dma.status.name)
        cur_consume = (dma.status is DMAStatus.CONSUME)
        if cur_consume:
            dma_rdy = True
            
        if i == grant_idx:
            dma.update(grant_ok=True)
        else:
            dma.update(grant_ok=False)

        if cur_consume and dma.status is DMAStatus.IDLE:
            consume_nxt = True
            
        if dma.status is not DMAStatus.IDLE:
            all_idle = False

    if DMAs[grant_idx].status is DMAStatus.FULL:
        if grant_idx == OSTD - 1:
            grant_idx = 0
        else:
            grant_idx = grant_idx + 1

    if consume_nxt:
        if consume_idx == OSTD - 1:
            consume_idx = 0
        else:
            consume_idx = consume_idx + 1

    if dma_rdy:
        rdy_cycle = rdy_cycle + 1

    if cmd == 0 and all_idle:
        break

    logging.debug('====================================================')
print('total aclk cycle:', cycle)
print('total nclk cycle:', round(cycle*NCLK/ACLK))
print('DMA ready %:', rdy_cycle / cycle * 100 , '%')

total aclk cycle: 5703
total nclk cycle: 7129
DMA ready %: 59.47746799929862 %
